In [2]:
import pandas as pd
link = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-hourly&stations=72495723213&dataTypes=DATE,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP&startDate=2000-01-01&endDate="
endDate = "2000-01-02"
link += endDate
data = pd.read_csv(link)
print(data)

        STATION                 DATE  SOURCE REPORT_TYPE CALL_SIGN  \
0   72495723213  2000-01-01T00:53:00       4       FM-15     KSTS    
1   72495723213  2000-01-01T01:53:00       4       FM-15     KSTS    
2   72495723213  2000-01-01T02:53:00       4       FM-15     KSTS    
3   72495723213  2000-01-01T03:53:00       4       FM-15     KSTS    
4   72495723213  2000-01-01T04:08:00       4       FM-16     KSTS    
..          ...                  ...     ...         ...       ...   
68  72495723213  2000-01-02T19:53:00       4       FM-15     KSTS    
69  72495723213  2000-01-02T20:53:00       4       FM-15     KSTS    
70  72495723213  2000-01-02T21:53:00       4       FM-15     KSTS    
71  72495723213  2000-01-02T22:53:00       4       FM-15     KSTS    
72  72495723213  2000-01-02T23:53:00       4       FM-15     KSTS    

   QUALITY_CONTROL          CIG               DATE.1      DEW  \
0             V020  22000,1,9,N  2000-01-01T00:53:00  +0070,1   
1             V020  22000,1,9